Для работы необходимо загрузить следующие файлы:

<br>model.pkl
<br>vectorizer.pickle
<br>emotion_text.py
<br>anger.wav

# Оглавление

[1. Импорт модулей](#Импорт-модулей)
<br>[2. Распознавание файла и конвертация в текст](#Распознавание-файла-и-конвертация-в-текст)
<br>[3. Загрузка модели из файла](#Загрузка-модели-из-файла)
<br>[4. Предсказание эмоции для заданного текста](#Предсказание-эмоции-для-заданного-текста)
<br>[5. Загрузка датафрейма](#Загрузка-датафрейма)
<br>[6. Деление на тренировочную, валидационную и тестовую выборки](#Train,-test,-valid)
<br>[7. Лемматизция текста](#Лемматизация-текста)
<br>[8. Удаление стоп-слов](#Удаление-стоп-слов)
<br>[9. Преобразование в TF-IDF вектора](#Преобразование-в-TF-IDF-вектора)
<br>[10. Обучение модели логистической регрессии](#Обучение-модели-логистической-регрессии)
<br>[11. Запись модели в файл](#Запись-модели-в-файл)



## Импорт модулей

In [31]:
!pip install pydub
!apt-get install ffmpeg
!pip install SpeechRecognition


from pydub import AudioSegment
import io
import speech_recognition as sr
import os  # Импортируем модуль os

import pickle
from tqdm import tqdm
from emotion_text import *


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 23.0 MB/s eta 0:00:00


## Распознавание файла и конвертация в текст

In [33]:
# Конвертация MP3 в WAV
# Ваш MP3-файл должен быть предварительно загружен в Colab
mp3_file = '/content/anger.mp3' # Укажите путь к своему файлу
audio = AudioSegment.from_mp3(mp3_file)

# Конвертация в WAV
wav_file = mp3_file.split('.')[0] + '.wav'
audio.export(wav_file, format='wav')

<_io.BufferedRandom name='/content/anger.wav'>

In [34]:
# Инициализация распознавателя
recognizer = sr.Recognizer()

# Загрузка WAV-файла
audio = AudioSegment.from_wav(wav_file)

# Задаем длительность фрагмента для распознавания (30 сек)
duration_in_milliseconds = len(audio)
fragment_size = 30 * 1000  # 30 секунд

# Инициализация пустой строки для собранного текста
full_text = ""

# Разбиение аудио и распознавание каждого фрагмента
for start in range(0, duration_in_milliseconds, fragment_size):
    end = min(start + fragment_size, duration_in_milliseconds)
    audio_fragment = audio[start:end]

    # Экспорт фрагмента во временный файл
    fragment_file = f"fragment_{start}_{end}.wav"
    audio_fragment.export(fragment_file, format="wav")

    # Распознавание фрагмента
    with sr.AudioFile(fragment_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='ru-RU')
            full_text += text + " "
        except sr.UnknownValueError:
            print(f"Фрагмент {start}-{end}: речь не распознана.")
        except sr.RequestError as e:
            print(f"Фрагмент {start}-{end}: ошибка запроса; {e}")

    # Удаление временных файлов
    os.remove(fragment_file)

# Вывод всего распознанного текста
print (len(full_text))
print("Распознанный текст:")
print(full_text)

89
Распознанный текст:
I'm so Angry that I feel like I have to go to life not been As Care Free As I want to be 


## Загрузка модели из файла

In [ ]:
with open('vectorizer.pickle', 'rb') as f:
    model_vec = pickle.load(f)

In [ ]:
with open('/content/model.pkl', 'rb') as ff:
    model = pickle.load(ff)

## Предсказание эмоции для заданного текста

In [ ]:
word = full_text

word_tfidf = vectorizer.transform([word])
vanga = model.predict(word_tfidf)

In [ ]:
vanga_answers(vanga)

You are anger, be happy!


_____________________________________________________________________

## Загрузка датафрейма

In [5]:
df = open_file('data.jsonl')

                                                text  label
0  i feel awful about it too because it s my job ...      0


In [ ]:
#df_sampl = df.head(100).copy()

## Train, test, valid
Делим датасет на тренировочную, валидационную и тестовую выборки

In [21]:
X_train, y_train, X_test, X_val, y_test, y_val = train_test_val(df.text, df.label)

## Лемматизация текста

In [22]:
X_train, X_test, X_val = lemmatize_text(X_train, X_test, X_val)

169233    i once again fail to play it i ll doubtless st...
Name: text, dtype: object


## Удаление стоп-слов

In [23]:
X_train, X_test, X_val = stopwords_text(X_train, X_test, X_val, language = 'english', num_row=1)

169233    fail play doubtless still feel jealous
Name: text, dtype: object


## Преобразование в TF-IDF вектора

In [24]:
X_train_tfidf, X_test_tfidf, X_val_tfidf, vectorizer = tf_idf_vec(X_train, X_test, X_val)

## Запись векторайзера в файл

In [25]:
with open("vectorizer.pickle", "wb") as ff:
    pickle.dump(vectorizer, ff)

## Обучение модели логистической регрессии

In [ ]:
for C in tqdm(np.arange(0.1, 5.0, 0.1)):

    model_LogisticRegression_L2 = LogisticRegression(class_weight='balanced',
                                                     multi_class='multinomial',
                                                     max_iter = 1000,
                                                     solver='lbfgs',
                                                     penalty='l2',
                                                     C=C)
    learning_model(X_train_tfidf, y_train, X_val_tfidf, y_val, X_test_tfidf, y_test, model_LogisticRegression_L2, C)
print_score(max_f1_test, max_f1_valid, max_acc_test, max_acc_valid)

100%|██████████| 49/49 [1:44:37<00:00, 128.11s/it]

Максимальный f1_score = 0.8999273590179936, полученный при значении С = 0.9 на тестовой выборке
Максимальный f1_score = 0.9010047840724188, полученный при значении С = 0.8 на валидационной выборке
Максимальный accuracy = 0.8971113936805739, полученный при значении С = 0.9 на тестовой выборке
Максимальный accuracy = 0.8982629975288501, полученный при значении С = 0.8 на валидационной выборке


## Запись модели в файл

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model_LogisticRegression_L2, f)